In [1]:
load_model = 'vgg16_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


## Load the Adamax optimizer based vgg16 model and test it

In [3]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False

vgg16.classifier = nn.Sequential(*list(vgg16.classifier.children())[:-1] + [nn.Linear(in_features=4096, out_features=120, bias=True)])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16 = vgg16.to(device)

vgg16.load_state_dict(torch.load(load_model), strict=False)
vgg16.eval()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
model = vgg16

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

per_class_accuracy

{'Pomeranian': 95.45454545454545,
 'basset': 88.88888888888889,
 'Japanese spaniel': 100.0,
 'Bouvier des Flandres': 80.0,
 'German shepherd': 87.5,
 'Brabancon griffon': 100.0,
 'Blenheim spaniel': 100.0,
 'kelpie': 81.25,
 'Shih Tzu': 90.9090909090909,
 'curly coated retriever': 87.5,
 'Lakeland terrier': 70.0,
 'Irish setter': 100.0,
 'German short haired pointer': 100.0,
 'flat coated retriever': 75.0,
 'Irish water spaniel': 93.33333333333333,
 'Dandie Dinmont': 88.88888888888889,
 'clumber': 100.0,
 'Yorkshire terrier': 82.35294117647058,
 'schipperke': 93.75,
 'briard': 75.0,
 'miniature poodle': 62.5,
 'chow': 100.0,
 'Saluki': 95.0,
 'boxer': 81.25,
 'Pekinese': 80.0,
 'komondor': 100.0,
 'Brittany spaniel': 87.5,
 'black and tan coonhound': 68.75,
 'Appenzeller': 43.75,
 'Eskimo dog': 20.0,
 'giant schnauzer': 75.0,
 'Old English sheepdog': 94.11764705882352,
 'Chihuahua': 87.5,
 'miniature schnauzer': 93.75,
 'redbone': 66.66666666666666,
 'wire haired fox terrier': 87.5,
 '

In [6]:
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

83.74485108115138

### Top 10 Breeds with least accuracy

In [7]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 20.0),
 ('Staffordshire bullterrier', 37.5),
 ('Appenzeller', 43.75),
 ('Cardigan', 50.0),
 ('Siberian husky', 50.0),
 ('Walker hound', 56.25),
 ('Lhasa', 57.89473684210527),
 ('Norwich terrier', 57.89473684210527),
 ('Border collie', 60.0),
 ('miniature poodle', 62.5)]

### Top 15 Breeds with most accuracy

In [8]:
k = 15
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Brabancon griffon', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Irish setter', 100.0),
 ('German short haired pointer', 100.0),
 ('clumber', 100.0),
 ('chow', 100.0),
 ('komondor', 100.0),
 ('Pembroke', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0),
 ('Rottweiler', 100.0),
 ('Kerry blue terrier', 100.0),
 ('groenendael', 100.0),
 ('basenji', 100.0)]